In [ ]:
pip install yfinance pandas numpy matplotlib scikit-learn statsmodels cvxpy torch

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import cvxpy as cp
import cylp
import matplotlib.pyplot as plt

# 1. Ações escolhidas (teste)
tickers = [
    'AAPL', 'MSFT', 'AMZN', 'GOOGL', 'NVDA', 'META', 'JNJ', 'V', 'JPM', 'UNH',
    'HD', 'PG', 'MA', 'PEP', 'BAC', 'DIS', 'AVGO', 'KO', 'PFE', 'TMO'
]
benchmark = '^OEX'  # Índice oficial do S&P 100
start_date = '2017-01-01'
end_date = '2023-12-31'
k = 20  # número de ações para selecionar por ano

# 2. Baixar dados ajustados de preço

data = yf.download(tickers + [benchmark], start=start_date, end=end_date)['Close']
returns = data.pct_change().dropna()

# 3. Lista de anos disponíveis nos dados
years = sorted(set(returns.index.year))

# 4. Armazenar resultados por ano
annual_results = []

for year in years:
    print(f"\n Ano: {year}")

    # Filtrar dados do ano
    year_data = returns[returns.index.year == year]

    # Selecionar as ações com maior variância dos retornos (se quiser diminuir o número de ações na carteira)
    variances = year_data[tickers].var()
    top_k = variances.sort_values(ascending=False).head(k).index.tolist()

    X = year_data[top_k].values
    y = year_data[benchmark].values

    # Número de observações no ano
    T = X.shape[0]

    # Otimização com cvxpy para minimizar o erro médio quadrático
    w = cp.Variable(k)
    objective = cp.Minimize((1/T) * cp.sum_squares(X @ w - y))
    constraints = [cp.sum(w) == 1, w >= 0]
    prob = cp.Problem(objective, constraints)
    prob.solve()

    weights = w.value
    portfolio_returns = X @ weights

    # Retorno acumulado do ano
    portfolio_cum = (1 + pd.Series(portfolio_returns, index=year_data.index)).cumprod()
    benchmark_cum = (1 + year_data[benchmark]).cumprod()

    # Guardar dados para treinar a rede neural
    X_mean = X.mean(axis=0)
    X_std = X.std(axis=0)
    inputs = np.concatenate([X_mean, [y.mean(), y.std()]])  # exemplo simples de features
    targets = weights
    if 'train_X' not in locals():
        train_X, train_y = [], []
    train_X.append(inputs)
    train_y.append(targets)

    # Resultados do ano
    annual_results.append({
        'year': year,
        'tickers': top_k,
        'weights': weights,
        'portfolio_cum': portfolio_cum,
        'benchmark_cum': benchmark_cum,
        'portfolio_ret': portfolio_cum.iloc[-1] - 1,
        'benchmark_ret': benchmark_cum.iloc[-1] - 1,
    })

# 5. Resultados por ano
for result in annual_results:
    year = result['year']
    print(f"\n {year}:")
    print(f"Retorno Carteira: {result['portfolio_ret']:.2}")
    print(f"Retorno do Índice (S&P 100): {result['benchmark_ret']:.2}")

    print(" Pesos otimizados:")
    for ticker, weight in zip(result['tickers'], result['weights']):
        print(f"  {ticker}: {weight:.2}")

    # Desempenho acumulado no ano
    plt.figure(figsize=(14,5))
    plt.plot(result['portfolio_cum'], label='Carteira Otimizada')
    plt.plot(result['benchmark_cum'], label='S&P 100 Index (^OEX)', linestyle='--')
    plt.title(f'Desempenho no ano {year}')
    plt.xlabel('Data')
    plt.ylabel('Retorno')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# 6. Tabela resumo geral dos anos
print("\n Comparação geral dos anos:")
df_summary = pd.DataFrame([
    {'Ano': r['year'], 'Carteira': r['portfolio_ret'], 'S&P 100': r['benchmark_ret']}
    for r in annual_results
])
df_summary['Excesso de Retorno'] = df_summary['Carteira'] - df_summary['S&P 100']
print(df_summary.to_string(index=False, float_format='{:,.2}'.format))


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

train_X = torch.tensor(np.array(train_X), dtype=torch.float32)
train_y = torch.tensor(np.array(train_y), dtype=torch.float32)

# Definir rede simples
class PortfolioNN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, output_dim),
            nn.Softmax(dim=1)  # força pesos somarem 1
        )
    def forward(self, x):
        return self.net(x)

model = PortfolioNN(train_X.shape[1], train_y.shape[1])
optimizer = optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

# Treinamento
for epoch in range(500):
    optimizer.zero_grad()
    pred = model(train_X)
    loss = loss_fn(pred, train_y)
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print(f"Epoch {epoch} | Loss: {loss.item():.6f}")


In [ ]:
# 1. Ações escolhidas (teste)
tickers = [
    'AAPL', 'MSFT', 'AMZN', 'GOOGL', 'NVDA', 'META', 'JNJ', 'V', 'JPM', 'UNH',
    'HD', 'PG', 'MA', 'PEP', 'BAC', 'DIS', 'AVGO', 'KO', 'PFE', 'TMO'
]
benchmark = '^OEX'  # Índice oficial do S&P 100
start_date = '2014-01-01'
end_date = '2017-12-31'
k = 20  # número de ações para selecionar por ano

# 2. Baixar dados ajustados de preço

data = yf.download(tickers + [benchmark], start=start_date, end=end_date)['Close']
returns = data.pct_change().dropna()

# 3. Lista de anos disponíveis nos dados
years = sorted(set(returns.index.year))
annual_results = {'nn': [], 'solver': []}

for i, year in enumerate(years):
    year_data = returns[returns.index.year == year]

    variances = year_data[tickers].var()
    top_k = variances.sort_values(ascending=False).head(k).index.tolist()
    X = year_data[top_k].values
    y = year_data[benchmark].values

    # Entrada para a rede neural
    X_features = np.concatenate([X.mean(axis=0), [y.mean(), y.std()]])
    X_tensor = torch.tensor(X_features, dtype=torch.float32).unsqueeze(0)
    pred_weights = model(X_tensor).detach().numpy().flatten()

    # Gerar retorno da carteira com pesos preditos
    portfolio_returns_nn = X @ pred_weights
    portfolio_cum_nn = (1 + pd.Series(portfolio_returns_nn, index=year_data.index)).cumprod()
    benchmark_cum_nn = (1 + year_data[benchmark]).cumprod()

    annual_results['nn'].append({
        'year': year,
        'tickers': top_k,
        'weights': pred_weights,
        'portfolio_cum': portfolio_cum_nn,
        'benchmark_cum': benchmark_cum_nn,
        'portfolio_ret': portfolio_cum_nn.iloc[-1] - 1,
        'benchmark_ret': benchmark_cum_nn.iloc[-1] - 1,
    })

    variances = year_data[tickers].var()
    top_k = variances.sort_values(ascending=False).head(k).index.tolist()
    X = year_data[top_k].values
    y = year_data[benchmark].values

    # Número de observações no ano
    T = X.shape[0]

    # Otimização com cvxpy para minimizar o erro médio quadrático
    w = cp.Variable(k)
    objective = cp.Minimize((1/T) * cp.sum_squares(X @ w - y))
    constraints = [cp.sum(w) == 1, w >= 0]
    prob = cp.Problem(objective, constraints)
    prob.solve()

    weights = w.value
    portfolio_returns_s = X @ weights

    # Retorno acumulado do ano
    portfolio_cum_s = (1 + pd.Series(portfolio_returns_s, index=year_data.index)).cumprod()
    benchmark_cum_s = (1 + year_data[benchmark]).cumprod()

    annual_results['solver'].append({
        'year': year,
        'tickers': top_k,
        'weights': weights,
        'portfolio_cum': portfolio_cum_s,
        'benchmark_cum': benchmark_cum_s,
        'portfolio_ret': portfolio_cum_s.iloc[-1] - 1,
        'benchmark_ret': benchmark_cum.iloc[-1] - 1,
    })

for s, n in zip(annual_results['nn'], annual_results['solver']):
    year = s['year']
    plt.figure(figsize=(14, 5))
    plt.plot(s['portfolio_cum'], label='Carteira (Solver)')
    plt.plot(n['portfolio_cum'], label='Carteira (NN)')
    plt.plot(s['benchmark_cum'], label='S&P 100 Index (^OEX)', linestyle='--')
    plt.title(f'Desempenho no ano {year} — Solver vs NN')
    plt.xlabel('Data')
    plt.ylabel('Retorno')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()


df_summary = pd.DataFrame([
    {
        'Ano': s['year'],
        'Carteira (Solver)': s['portfolio_ret'],
        'Carteira (NN)': n['portfolio_ret'],
        'S&P 100': s['benchmark_ret']
    }
    for s, n in zip(annual_results['solver'], annual_results['nn'])
])

df_summary['Excesso Solver'] = df_summary['Carteira (Solver)'] - df_summary['S&P 100']
df_summary['Excesso NN'] = df_summary['Carteira (NN)'] - df_summary['S&P 100']

print("\n=== Comparação geral dos anos ===")
print(df_summary.to_string(index=False, float_format='{:,.4}'.format))
